## Hent 'fersk' Databasefil
(Tips: https://www.geeksforgeeks.org/how-to-write-pandas-dataframes-to-multiple-excel-sheets/)

In [3]:
from spreadsheet_functions import fetchDB
fetchDB("develop") # Branches: develop, test, master (develop is the default, test is the test branch, master is the production branch)

db-file downloaded to path 'db/nin3kodeapi.db'


# Lag Excel
* Creates excel-document from downloaded sqlite database


In [1]:
from spreadsheet_functions import createExcel
createExcel()

Excel data skrevet til ut/nin3_0_20231211_214638.xlsx


# Type-fanen

In [ ]:
from spreadsheet_functions import type_fane, getConn,closeConn
display(type_fane(makecsv=True).head(3))
closeConn()

## Hovedtypegruppe-fanen

In [ ]:
#HTG
from spreadsheet_functions import hovedtypegruppe_fane,closeConn
display(hovedtypegruppe_fane(makecsv=True).head(3))
closeConn()

## Hovedtype-fanen

In [ ]:
from spreadsheet_functions import hovedtype_fane,closeConn
display(hovedtype_fane(makecsv=True).head(3))
closeConn()

## Grunntype-fanen

In [ ]:
from spreadsheet_functions import grunntype_fane,closeConn
display(grunntype_fane(makecsv=True).head(3))
closeConn() 

## Kartleggingsenheter M005

In [ ]:
from spreadsheet_functions import kle_m005,closeConn
display(kle_m005(makecsv=True).head(3))
closeConn()

## Kartleggingsenheter M020

In [ ]:
from spreadsheet_functions import kle_m020,closeConn
display(kle_m020(makecsv=True).head(3))
closeConn()

## Kartleggingsenheter M050

In [ ]:
from spreadsheet_functions import kle_m050,closeConn
display(kle_m050(makecsv=True).head(3))
closeConn()

## Hovedtype_KLE_M005

In [ ]:
from spreadsheet_functions import ht_kle_m005,closeConn
display(ht_kle_m005(makecsv=True).head(3))
closeConn()

## Hovedtype_KLE_M020

In [ ]:
from spreadsheet_functions import ht_kle_m020,closeConn
display(ht_kle_m020(makecsv=True).head(3))
closeConn()

## Hovedtype_KLE_M050

In [ ]:
from spreadsheet_functions import ht_kle_m050,closeConn
display(ht_kle_m050(makecsv=True).head(3))
closeConn()

## Grunntype_KLE_M005

In [ ]:
from spreadsheet_functions import gt_kle_m005,closeConn
display(gt_kle_m005(makecsv=True).head(3))
closeConn()

## Grunntype_KLE_M020

In [ ]:
from spreadsheet_functions import gt_kle_m020,closeConn
display(gt_kle_m020(makecsv=True).head(3))
closeConn()

## Grunntype_KLE_M050

In [ ]:
from spreadsheet_functions import gt_kle_m050,closeConn
display(gt_kle_m050(makecsv=True).head(3))
closeConn()

## Hovedtypegruppe_Hovedøkosystem

In [ ]:
from spreadsheet_functions import htg_hoek,closeConn
display(htg_hoek(makecsv=True).head(3))
closeConn()

## Variabel

In [ ]:
from spreadsheet_functions import variabel_fane,closeConn
display(variabel_fane(makecsv=True).head(3))
closeConn()

## Variabelnavn

In [ ]:
from spreadsheet_functions import variabel_fane,closeConn
display(variabel_fane(makecsv=True).head(3))
closeConn()

## Måleskala

In [ ]:
from spreadsheet_functions import maaleskala_fane,closeConn
display(maaleskala_fane(makecsv=True).head(3))
closeConn()

## Trinn

In [1]:
from spreadsheet_functions import trinn_fane,closeConn
display(trinn_fane(makecsv=True).head(3))
closeConn()

written to 'ut/trinn_fane_20231211_212630.csv'


,Verdi,Beskrivelse,MaaleskalaNavn
0,nan_nan,,nan-nan
1,KA_a,svært kalkfattig,KA-SO
2,HS_KB,plukkhogst,HS-SI


## Variabelnavn_Måleskala

In [3]:
from spreadsheet_functions import variabelnavn_maaleskala,closeConn  
display(variabelnavn_maaleskala(makecsv=True).head(3))
closeConn()

written to 'ut/variabelnavn_maaleskala_fane_20231211_212515.csv'


,VariabelnavnKode,Variabelnavn,MaaleskalaNavn
0,AD-FA,Fremmedartsantall,D0
1,AD-FD,Relativ fremmedartsdekning,P6b
2,AD-FR,Relativ fremmedartsandel,P6b


## Hovedtype_Variabeltrinn

In [1]:
from spreadsheet_functions import hovedtype_Variabeltrinn,closeConn
display(hovedtype_Variabeltrinn(makecsv=True).head(3))
closeConn()

written to 'ut/hovedtype_variabeltrinn_fane_20231211_212033.csv'


,GrunntypeKode,VariabelnavnKode,MaaleskalaNavn,TrinnVerdi
0,NF-A-01,None,HY-SI,HY_A
1,NF-A-02,None,HY-SI,HY_B
2,NF-B-01,None,HY-SI,HY_A


## Grunntype_Variabeltrinn

In [2]:
from spreadsheet_functions import grunntype_Variabeltrinn,closeConn
display(grunntype_Variabeltrinn(makecsv=True).head(3))
closeConn()

written to 'ut/grunntype_variabeltrinn_fane_20231211_212058.csv'


,GrunntypeKode,VariabelnavnKode,MaaleskalaNavn,TrinnVerdi
0,F-A-01-01,LM-HU,HU-SO,HU_0
1,F-A-01-01,LM-HU,HU-SO,HU_a
2,F-A-01-01,LM-KA,KA-SO,KA_a
